In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import joblib

# Compute engine stand alone

In [21]:
file = joblib.load('schemas.barfi')

In [22]:
from_client = joblib.load('_from_client.pkl')

In [23]:
from blocks import init
from streamlit import session_state as session
session.dados = pd.DataFrame()
init()

starting blocks
colums: RangeIndex(start=0, stop=0, step=1)


In [24]:
from Barfi import ComputeEngine

def get_execution(fluxo, base_blocks_list):
    _ce = ComputeEngine(blocks=base_blocks_list)
    _ce.add_editor_state(fluxo)
    _ce._map_block_link()
    _ce._execute_compute()

In [25]:
get_execution(file['soma3'], session.blocks)

Resultado: 3


# Entrada block

In [26]:
file = joblib.load('schemas.barfi')
entr = file['entradas']

In [27]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [28]:
saidas = [i for i in entr['nodes'] if i['type'] == 'saida']
saida_names = [i['name'] for i in saidas]
entradas = [i for i in entr['nodes'] if i['type'] == 'entrada']
entrada_names = [i['name'] for i in entradas]

In [29]:
def get_dict_from_interfaces(interface):
    aux = {}
    for i in interface:
        aux[i[0]] = i[1]
    return aux

def encode_dict_to_interface(aux_dict):
    interface = [[key, aux_dict[key]] for key in aux_dict]
    return interface

def encode_val_to_entradas(entra, val_dict):
    aux = entra.copy()
    for entrada in aux:
        name = entrada['name']
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        tmp['entrada']['value'] = val_dict[name]
        encoded_interface = encode_dict_to_interface(tmp)
        entrada['interfaces'] = encoded_interface
    return aux

def get_ids_entradas(entradas):
    aux = entradas.copy()
    ids = []
    for entrada in aux:
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        ids.append(tmp['entrada']['id'])
    return ids

def get_ids_from_conn(connections, to_ids):
    aux = connections.copy()
    entry_dict = {}
    ids = []
    ids_from = []
    pos = []
    for i, conn in enumerate(connections):
        if conn['to'] in to_ids:
            entry_dict[i] = {'id': conn['id'],
                             'from': conn['from'],
                             'to': conn['to']}
    return entry_dict

In [30]:
val_dict = {'entrada-1': 2, 'entrada-2': 3}
tmp = entradas.copy()
new_entr = encode_val_to_entradas(tmp, val_dict)

In [31]:
ids_entradas = get_ids_entradas(entradas)
entry_conn = get_ids_from_conn(entr['connections'], ids_entradas)

## Antes de cortar conn:

In [32]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


## Pos cortar conn

In [40]:
new_entr = entr.copy()
pos_list = [i for i in entry_conn.keys()]
pos_list.sort()
pos_list = pos_list[::-1]
for i in pos_list:
    new_entr['connections'].pop(i)

In [41]:
get_execution(new_entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 1
Resultado: 2


In [42]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 1
Resultado: 2


In [49]:
new_entr['nodes'][2]['interfaces'][0][1]['value']

2